## 기존의 xlsx 파일을 csv로 변환

### 전처리가 필요한 이유

    크롤링 과정에서 다음 가사의 html 태그와 19금 노래를 지우지 못한 채 가져온 것.

    크롤러를 실 서비스에서 작동시킨다면, 이를 해결하고,

    앨범 재킷과 아티스트 이미지 등을 어떻게 처리할지 생각해볼 필요가 있다.

In [1]:
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm

song_df = pd.read_excel('../song_data/song_xlsx/song_df.xlsx', index_col=0)
artist_df = pd.read_excel('../song_data/song_xlsx/artist_df.xlsx', index_col=0)
album_df = pd.read_excel('../song_data/song_xlsx/album_df.xlsx', index_col=0)
song_artist_df = pd.read_excel('../song_data/song_xlsx/song_artist_df.xlsx', index_col=0)
song_album_df = pd.read_excel('../song_data/song_xlsx/song_album_df.xlsx', index_col=0)
song_genre_df = pd.read_excel('../song_data/song_xlsx/song_genre_df.xlsx', index_col=0)
lyrics_df = pd.read_excel('../song_data/song_xlsx/lyrics_df.xlsx', index_col=0)

### 장르 테이블 (gnere_id, genre_name) 생성

In [7]:
genre_df = pd.DataFrame(columns=['genre_id', 'genre_name'])

genre_list = [
    [101, '발라드'],
    [102, '포크/블루스'],
    [103, '성인가요/트로트'],
    [104, '록/메탈'],
    [105, '국내영화'],
    [106, '댄스'],
    [107, '국내드라마'],
    [108, '뉴에이지'],
    [109, '클래식'],
    [110, '크로스오버'],
    [111, '교향/관현악'],
    [112, '랩/힙합'],
    [113, '-'],
    [114, '재즈'],
    [115, '보컬재즈'],
    [116, '인디음악'],
    [117, 'R&B/Soul'],
    [118, '애시드/퓨전/팝'],
    [119, '애니메이션/웹툰'],
    [120, 'POP'],
    [121, '일렉트로니카'],
    [122, 'J-POP'],
    [123, 'CCM'],
    [124, '국내CCM'],
    [125, '워십'],
    [126, '찬송가'],
    [127, '게임'],
    [128, '키즈'],
    [129, '만화'],
    [130, '국외영화'],
    [131, '국내뮤지컬']
]

for x,y in genre_list:
    genre_df.loc[len(genre_df)] = [x, y]

genre_df.index = genre_df.index +1
genre_df

genre_df.to_csv('../song_data/genre.csv', encoding='utf-8')

,genre_id,genre_name
2,101,발라드
3,102,포크/블루스
4,103,성인가요/트로트
5,104,록/메탈
6,105,국내영화
7,106,댄스
8,107,국내드라마
9,108,뉴에이지
10,109,클래식
11,110,크로스오버


### xlsx -> csv

#### 메인 노래 테이블

In [8]:
song_df = song_df.drop_duplicates(['song_id']).reset_index(drop=True)
for i in tqdm(range(len(song_df))):
    title = song_df.loc[i, 'title']
    # 타이틀의 , -> & 로 변환
    song_df.loc[i, 'title'] = title.replace(',', '&')
    # 19금 노래는 가사를 받아올 수 없으므로 nan
    if title[:3] == '19금':
        title = np.nan
        song_df.loc[i, 'title'] = title

# 19금 노래 제거 후 인덱스 리셋
song_df = song_df.dropna(axis=0).reset_index(drop=True)
song_df.index += 1

# csv로 저장
song_df.to_csv('../song_data/song.csv', encoding='UTF-8-SIG')

  0%|          | 0/9278 [00:00<?, ?it/s]

#### 노래 - [아티스트, 앨범, 장르] 연결 테이블

In [5]:
# 노래 테이블에 song_id가 없는 연결 삭제

def drop_not_in_song(df, name):
    df = df.drop_duplicates().reset_index(drop=True)
    for i in tqdm(range(len(df))):
        song_id = df.loc[i, 'song_id']
        if not song_df[song_df.loc[:, 'song_id'] == song_id].song_id.any():
            df.loc[i, 'song_id'] = np.nan

    df = df.dropna(axis=0).reset_index(drop=True).astype(int)
    df.index += 1

    df.to_csv(f'../song_data/{name}.csv', encoding='utf-8-sig')

drop_not_in_song(song_artist_df, 'song_artist')
drop_not_in_song(song_genre_df, 'song_genre')
drop_not_in_song(song_album_df, 'song_album')

  0%|          | 0/9200 [00:00<?, ?it/s]

#### 가사는 집중 처리 (가사 내부의 html태그, \n 제거)

In [10]:
for i in tqdm(range(len(lyrics_df))):
    lyrics = lyrics_df.loc[i, 'lyrics']
    
    while '<' in lyrics:
        lyrics = lyrics[:lyrics.index('<')] + lyrics[lyrics.index('>')+1:]
    
    lyrics = lyrics.replace('\n', '').replace('   ', ' ').replace('  ', '')
    
    if lyrics == 'None':
        lyrics = np.nan
    
    song_id = lyrics_df.loc[i, 'song_id']
        
    if not song_df[song_df.loc[:, 'song_id'] == song_id].song_id.any():
        lyrics_df.loc[i, 'song_id'] = np.nan
        
    lyrics_df.loc[i, 'lyrics'] = lyrics

lyrics_df = lyrics_df.dropna(axis=0).reset_index(drop=True)
lyrics_df.index += 1
lyrics_df.to_csv('../song_data/lyrics.csv', encoding='utf-8')

  0%|          | 0/9278 [00:00<?, ?it/s]

#### 앨범 테이블 처리 ( 이미지 path 수정)

In [6]:
album_df = album_df.drop_duplicates('album_id').reset_index(drop=True)
for i in tqdm(range(len(album_df))):
    
    url = album_df.loc[i, 'album_img_path']
    
    idx = url.find('jpg?')
    if idx+1:
        idx += 3
        url = url[:idx]
    idx = url.find('/melon/')
    if idx+1:
        url = url[:idx]
    
    album_df.loc[i, 'album_img_path'] = url
    
    album_id = album_df.loc[i, 'album_id']
    
    if not song_album_df[song_album_df.loc[:, 'album_id'] == album_id].album_id.any():
        album_df.loc[i, 'album_title'] = np.nan
    
album_df = album_df.dropna(axis=0).reset_index(drop=True)
album_df.index += 1
album_df.to_csv('../song_data/album.csv', encoding='utf-8-sig')

  0%|          | 0/4170 [00:00<?, ?it/s]

#### Artist 수정

In [109]:
artist_df = artist_df.drop_duplicates('artist_id').reset_index(drop=True)
for i in tqdm(range(len(artist_df))):
    
    url = artist_df.loc[i, 'artist_img_path']
    
    idx = url.find('jpg?')
    if idx+1:
        idx += 3
        url = url[:idx]
    idx = url.find('/melon/')
    if idx+1:
        url = url[:idx]
    
    artist_df.loc[i, 'artist_img_path'] = url
    
artist_df = artist_df.dropna(axis=0).reset_index(drop=True)

artist_df.to_csv('../song_data/artist.csv', encoding='utf-8')

  0%|          | 0/2980 [00:00<?, ?it/s]

In [12]:
group_artist_df = pd.read_excel('../song_data/song_xlsx/group_artist_df.xlsx', index_col=0)
group_artist_df.index += 1
group_artist_df.to_csv('../song_data/group_artist.csv', encoding='utf-8')

In [13]:
artist_df = pd.read_excel('../song_data/song_xlsx/artist_df.xlsx', index_col=0)
artist_df.index += 1
artist_df.to_csv('../song_data/artist.csv', encoding='utf-8')

### 가사 분석용 데이터 셋 생성

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
df = pd.DataFrame(columns=['song_id', 'title', 'genre', 'lyrics'])

lyrics_df = pd.read_csv('../song_data/lyrics.csv', index_col=0)
song_df = pd.read_csv('../song_data/song.csv', index_col=0, encoding='cp949')
song_genre_df = pd.read_csv('../song_data/song_genre.csv', index_col=0)
genre_df = pd.read_csv('../song_data/genre.csv', index_col=0)


In [4]:
for i in tqdm(range(len(lyrics_df))):
    song_id = lyrics_df.loc[i, 'song_id']
    lyrics = lyrics_df.loc[i, 'lyrics']
    title = song_df[song_df.loc[:, 'song_id'] == song_id].title.item()
    song_genre = []
    genre_list = [g for g in song_genre_df[song_genre_df.loc[:, 'song_id'] == song_id].genre_name]
    for g in genre_list:
        song_genre.append(genre_df[genre_df['genre_id'] == g].genre_name.item())
    '&'.join(song_genre)
    
    df.loc[i, :] = [song_id, title, song_genre, lyrics]

df.to_csv('../song_data/learn_song_lyrics.csv')

  0%|          | 0/9134 [00:00<?, ?it/s]